In [24]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv('../data/cleaned.csv', index_col=0)
df.head()

,Duration,Credit Amount,Installment rate,Residence,Age,Number of credits,Maintenance,Target,Account Status_<0,Account Status_<200,...,Housing_own,Housing_rent,Job_management/ highly qualified employee,Job_skilled employee / official,Job_unemployed/ unskilled - non-resident,Job_unskilled - resident,Telephone_none,Telephone_yes,Foreign_no,Foreign_yes
0,6,1169,4,4,67,2,1,1,1,0,...,1,0,0,1,0,0,0,1,0,1
1,48,5951,2,2,22,1,1,0,0,1,...,1,0,0,1,0,0,1,0,0,1
2,12,2096,2,3,49,1,2,1,0,0,...,1,0,0,0,0,1,1,0,0,1
3,42,7882,2,4,45,1,2,1,1,0,...,0,0,0,1,0,0,1,0,0,1
4,24,4870,3,4,53,2,2,0,1,0,...,0,0,0,1,0,0,1,0,0,1


Here I use 5 fold CV.

In [3]:
Xs = df.loc[:, df.columns != 'Target']
ys = df['Target']

all_preds = np.zeros(df.shape[0])

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(df):
    X_train, y_train = Xs.iloc[train_index], ys.iloc[train_index]
    X_test, y_test = Xs.iloc[test_index], ys.iloc[test_index]
    
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    
    all_preds[test_index] = lr.predict(X_test)
#     print(accuracy_score(y_test, preds))

accuracy_score(ys, all_preds)

0.739

In [4]:
orig_df = pd.read_csv('../data/orig.csv', index_col=0)
orig_df.rename(columns={"Target": "label_value"}, inplace=True)
orig_df['score'] = all_preds
orig_df.head()

,Account Status,Duration,Credit History,Purpose,Credit Amount,Savings,Employment,Installment rate,Sex,Other debtors,...,Age,Other installments,Housing,Number of credits,Job,Maintenance,Telephone,Foreign,label_value,score
0,<0,6,critical account,radio/television,1169,no,>= 7 years,4,male single,none,...,67,none,own,2,skilled employee / official,1,yes,yes,1,1.0
1,<200,48,existing credits paid back duly till now,radio/television,5951,<100,1 <= < 4 years,2,female divorced/separated/married,none,...,22,none,own,1,skilled employee / official,1,none,yes,0,0.0
2,no,12,critical account,education,2096,<100,4 <= < 7 years,2,male single,none,...,49,none,own,1,unskilled - resident,2,none,yes,1,1.0
3,<0,42,existing credits paid back duly till now,furniture/equipment,7882,<100,4 <= < 7 years,2,male single,guarantor,...,45,none,for free,1,skilled employee / official,2,none,yes,1,1.0
4,<0,24,delay in paying off,car (new),4870,<100,1 <= < 4 years,3,male single,none,...,53,none,for free,2,skilled employee / official,2,none,yes,0,0.0


In [5]:
orig_df.to_csv('../data/processed.csv')

In [10]:
orig_df['label_value'].value_counts()

1    700
0    300
Name: label_value, dtype: int64

In [41]:
Xs = df.loc[:, df.columns != 'Target']
ys = df['Target']

remove_special = lambda string: string.replace("<", "")

Xs.columns = map(remove_special, Xs.columns)

clf = XGBClassifier()
kfold = KFold(n_splits=10)
results = cross_val_score(clf, Xs, ys, cv=kfold)
print(f"Accuracy: {results.mean()} ({results.std()})")

Accuracy: 0.763 (0.037161808352124126)
